In [ ]:
import pandas as pd
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
import numpy as np
from collections import Counter
user_history = pd.read_csv('user_history.csv')
user_ratings = pd.read_csv('user_ratings.csv')

In [ ]:
history_by_id = user_history.set_index('USER ID')
al_bas = user_ratings.groupby('PRODUCT').get_group('balance hostel').set_index('USER ID').drop(['PRODUCT'],axis=1)
al_bas_users_hist = history_by_id.loc[al_bas.index]

In [ ]:
al_bas_users_hist['RATING'] = al_bas

In [ ]:
num = al_bas_users_hist.shape[0]
al_bas_users_hist.shape

In [ ]:
al_bas_users_hist = al_bas_users_hist.sample(frac=1)
train_frame = al_bas_users_hist.head(1200)
test_frame = al_bas_users_hist.tail(num-1200)

print(train_frame.shape,test_frame.shape)

In [ ]:
X_test = test_frame.drop(['RATING'], axis=1).to_numpy()
y_test = np.array([[v] for v in list(test_frame.RATING)])

In [ ]:
maxm = train_frame.RATING.value_counts().max()
indx_srtd = train_frame.RATING.value_counts().sort_index()
indx_srtd

In [ ]:
row_list = []
for i in range(11):
    v = indx_srtd[i]
    g_i = train_frame.groupby('RATING').get_group(i)
    while v<maxm:
        for index, row in g_i.iterrows():
            row_list.append(row)
            v+=1
            
            if v>=maxm:
                break

train_frame_augmented = train_frame.append(row_list)
train_frame_augmented = train_frame_augmented.sample(frac=1)#this is to shuffle the rows or else training will takes batches serially and fuck it up
y = list(train_frame_augmented.RATING)

In [ ]:
train_frame_augmented.RATING.value_counts()

In [ ]:
X_train = train_frame_augmented.drop(['RATING'], axis=1).to_numpy()

In [ ]:
y_train = np.array([[v] for v in y])

In [ ]:
#This is the right model, don't change this!
"""
model = Sequential()

model.add(Dense(100,activation='tanh'))
model.add(Dense(20,activation='tanh'))
model.add(Dense(1))

sgd = SGD(lr = 0.01, decay = 0.0, momentum=0.8, nesterov=True)
model.compile(loss='mae', optimizer='adam', metrics=['mse']) #mae=mean absolute error

model.fit(X, y, validation_split=0.2, epochs=20, batch_size=50)"""

In [ ]:
#Experiment here

model = Sequential()

model.add(Dense(100,activation='tanh'))
model.add(Dense(20,activation='tanh'))
model.add(Dense(1))

sgd = SGD(lr = 0.01, decay = 0.0, momentum=0.8, nesterov=True)
model.compile(loss='mae', optimizer='adam', metrics=['mse']) #mae=mean absolute error

model.fit(X_train, y_train, validation_split=0.0, epochs=20, batch_size=50)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
rounded = [int(round(v[0])) for v in y_pred]
y_new = [i[0] for i in y_test]
diff = np.array([rounded[i]-y_new[i] for i in range(len(y_new))])

In [ ]:
i_true = Counter(y_new).items()
i_pred = Counter(rounded).items()

true = {k:0 for k in range(11)}
for key, value in i_true:
    true[key] = value

pred = {k:0 for k in range(11)}
for key, value in i_pred:
    pred[key] = value

for i in range(11):
    print('{} true:{}  pred:{}'.format(i,true[i],pred[i]))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(11),[true[x] for x in range(11)],label='true')
plt.plot(range(11),[pred[x] for x in range(11)],label='pred')
plt.legend()
plt.show()

In [ ]:
Counter(diff)

In [ ]:
models = []
for i in range(1,10):
    X = train_frame.drop(['RATING'], axis=1).to_numpy()
    
    y = np.array([np.array(arr) for arr in train_frame.RATING.apply(lambda x: [int(x<i),int(x==i),int(x>i)])])
    
    models.append(Sequential())
    models[i-1].add(Dense(100,activation='tanh'))
    models[i-1].add(Dense(20,activation='tanh'))
    models[i-1].add(Dense(3,activation='softmax'))

    sgd = SGD(lr = 0.01, decay = 0.0, momentum=0.8, nesterov=True)
    models[i-1].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #mae=mean absolute error

    models[i-1].fit(X, y, validation_split=0.2, epochs=20, batch_size=50)
    
#interpret the predicted tupla as probability and take product to get the probability of being a particular value
#then take max of all the values to get the answer

In [ ]:
i = 0
X = train_frame.drop(['RATING'], axis=1).to_numpy()
y = np.array([np.array(arr) for arr in train_frame.RATING.apply(lambda x: [int(x==i),int(x>i)])])

model0 = Sequential()
model0.add(Dense(100,activation='tanh'))
model0.add(Dense(20,activation='tanh'))
model0.add(Dense(2,activation='softmax'))

sgd = SGD(lr = 0.01, decay = 0.0, momentum=0.8, nesterov=True)
model0.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #mae=mean absolute error

model0.fit(X, y, validation_split=0.2, epochs=20, batch_size=50)

#models = [model] + models

i = 10

X = train_frame.drop(['RATING'], axis=1).to_numpy()
    
y = np.array([np.array(arr) for arr in train_frame.RATING.apply(lambda x: [int(x<i),int(x==i)])])

model10 = Sequential()
model10.add(Dense(100,activation='tanh'))
model10.add(Dense(20,activation='tanh'))
model10.add(Dense(2,activation='softmax'))

sgd = SGD(lr = 0.01, decay = 0.0, momentum=0.8, nesterov=True)
model10.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #mae=mean absolute error

model10.fit(X, y, validation_split=0.2, epochs=20, batch_size=50)

#models = models + [model]

In [ ]:
X = test_frame.drop(['RATING'],axis=1).to_numpy()
y = list(test_frame.RATING)
model_preds = []
for model in models:
    model_preds.append(model.predict(X))

mod_preds0= model0.predict(X)
mod_preds10= model10.predict(X)

In [ ]:
def prob_calc(l):
    prob = np.ones(11,dtype=float)
    prob = prob * [l[0][int(i!=0)] for i in range(11)]
    
    prob = prob * [l[10][int(i==10)] for i in range(11)]
    for i in range(1,10):
        multiplier = [l[i][np.sign(j-i)+1] for j in range(0,11)]
        prob *= multiplier
        
    return prob

y_res = []

for i in range(len(X)):
    prob_v = prob_calc([model[i] for model in model_preds])
    y_res.append([list(prob_v).index(max(prob_v))])

In [446]:
rounded = [v[0] for v in y_res]

diff = np.array([rounded[i]-y[i] for i in range(len(y_new))])

In [447]:
i_true = Counter(y).items()
i_pred = Counter(rounded).items()

true = {k:0 for k in range(11)}
for key, value in i_true:
    true[key] = value

pred = {k:0 for k in range(11)}
for key, value in i_pred:
    pred[key] = value

for i in range(11):
    print('{} true:{}  pred:{}'.format(i,true[i],pred[i]))

0 true:6  pred:6
1 true:12  pred:9
2 true:16  pred:9
3 true:46  pred:18
4 true:67  pred:14
5 true:35  pred:151
6 true:30  pred:56
7 true:38  pred:10
8 true:21  pred:8
9 true:11  pred:2
10 true:8  pred:7


In [2]:
import numpy as np
sim = np.load('kmeans errors vs k.npy')

In [3]:
sim

array([[1.00000000e+00, 4.19847407e+05],
       [2.00000000e+00, 2.49727526e+05],
       [3.00000000e+00, 9.55217422e+04],
       [4.00000000e+00, 8.57410126e+04],
       [5.00000000e+00, 7.80306422e+04],
       [6.00000000e+00, 7.10940738e+04],
       [7.00000000e+00, 6.67769671e+04],
       [8.00000000e+00, 6.25645858e+04],
       [9.00000000e+00, 5.92732414e+04],
       [1.00000000e+01, 5.68982288e+04],
       [1.10000000e+01, 5.46418392e+04],
       [1.20000000e+01, 5.28923119e+04],
       [1.30000000e+01, 5.09724317e+04],
       [1.40000000e+01, 4.92794764e+04],
       [1.50000000e+01, 4.79063423e+04],
       [1.60000000e+01, 4.66712360e+04],
       [1.70000000e+01, 4.55794742e+04],
       [1.80000000e+01, 4.46120607e+04],
       [1.90000000e+01, 4.38079842e+04],
       [2.00000000e+01, 4.29712423e+04],
       [2.10000000e+01, 4.22574272e+04],
       [2.20000000e+01, 4.16312408e+04],
       [2.30000000e+01, 4.09288454e+04],
       [2.40000000e+01, 4.02885338e+04],
       [2.500000

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

user_history = pd.read_csv('user_history.csv')
user_ratings = pd.read_csv('user_ratings.csv')

In [ ]:
id_list = list(user_history['USER ID'])
user_ratings.groupby('USER ID')['PRODUCT'].apply(list).apply(len)
alaska_basil = user_ratings.groupby('PRODUCT')['USER ID'].apply(list)['pegasus respond']
history_by_id = user_history.set_index('USER ID')
al_bas = user_ratings.groupby('PRODUCT').get_group('pegasus respond').set_index('USER ID').drop(['PRODUCT'],axis=1)
al_bas_users_hist = history_by_id.loc[al_bas.index]
X = al_bas_users_hist.to_numpy()
y = al_bas.to_numpy()

In [ ]:
#DONT DELETE THIS
#THIS MODEL LEADS TO OVERFITTING

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD

model = Sequential()

model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(1))

sgd = SGD(lr = 0.05, decay = 0.0, momentum=0.8, nesterov=True)
model.compile(loss='mse', optimizer='adam', metrics=['mae']) #mae=mean absolute error

model.fit(X, y, validation_split=0.2, epochs=150, batch_size=50)

In [ ]:
#This is the right model, don't change this!

model = Sequential()

model.add(Dense(100,activation='tanh'))
model.add(Dense(20,activation='tanh'))
model.add(Dense(1))

sgd = SGD(lr = 0.01, decay = 0.0, momentum=0.8, nesterov=True)
model.compile(loss='mae', optimizer='adam', metrics=['mse']) #mae=mean absolute error

model.fit(X, y, validation_split=0.2, epochs=100, batch_size=50)

In [ ]:
y_val = model.predict(X)

In [ ]:
rounded = [int(round(v[0])) for v in y_val]
y_new = [i[0] for i in y]
diff = np.array([rounded[i]-y_new[i] for i in range(len(y_new))])

In [ ]:
Counter(diff)

In [ ]:
Counter(rounded)

In [ ]:
Counter(y_new)

In [ ]:
d ={key:0 for key in range(11)}
for key, value in Counter(rounded).items():
    d[abs(key)] = value
    
d

In [ ]:
e ={key:0 for key in range(11)}
for key, value in Counter(y_new).items():
    e[abs(key)] = value
e

In [ ]:
for i in range(len(y)):
    print(y[i][0],y_val[i][0])

In [ ]:
U, S, VT = np.linalg.svd(X.T)

In [ ]:
(S[0]**2+S[1]**2+S[2]**2+S[3]**2)/sum(S**2)

In [ ]:
M = np.random.rand(100,100)

In [ ]:
X_new = U.T@X.T

In [ ]:
X_new = X_new.T

In [ ]:
model = Sequential()

model.add(Dense(100,activation='tanh'))
model.add(Dense(20,activation='tanh'))
model.add(Dense(1))

sgd = SGD(lr = 0.01, decay = 0.0, momentum=0.8, nesterov=True)
model.compile(loss='mae', optimizer='adam', metrics=['mse']) #mae=mean absolute error

model.fit(X_new, y, validation_split=0.2, epochs=100, batch_size=50)

In [ ]:
y_val = model.predict(X_new)

In [ ]:
rounded = [int(round(v[0])) for v in y_val]
y_new = [i[0] for i in y]
diff = np.array([rounded[i]-y_new[i] for i in range(len(y_new))])

In [ ]:
Counter(diff)

In [ ]:
Counter(y_new)

In [ ]:

a_mult = X[:,:-1]         #extract the columns containing a values

b_mult = y
A_mult = np.vstack([a_mult.T, np.ones(a_mult.shape[0])]).T  #create the matrix A with a's as the rows and ones in the last column

result_mult = np.linalg.lstsq(A_mult, b_mult, rcond=None)
tup = result_mult[0]
x = tup[:-1]
c = tup[-1]
error_mult = result_mult[1]
print("The vector x is:", x)
print("c is:", c)
print("Total error is", error_mult[0])

In [ ]:
y_reg = A_mult@tup

In [ ]:
y_reg = [int(round(e[0])) for e in y_reg]

In [ ]:
rounded = [((v[0])) for v in y_reg]
y_new = [i[0] for i in y_val]
#diff = np.array([rounded[i]-y_new[i] for i in range(len(y_new))])

In [ ]:
Counter(diff)

In [ ]:
y_vag= (np.array(rounded)+np.array(y_new))/2

In [ ]:
def rounder(num):
    if num<4 or num>6:
        return round(num)
    else:
        if num<4.25:
            return 4
        else:
            return 6
y_rdd = [rounder(x) for x in y_vag]

In [ ]:
diff = np.array([y_rdd[i]-y[i] for i in range(len(y_rdd))])

In [ ]:
diff = [x[0] for x in diff]

In [ ]:
Counter(diff)

In [ ]:
y_vag